In [ ]:
# Fonte de dados - Serra - ES

# Nao funciona o download desta forma
# https://ioes.dio.es.gov.br/diariodaserra/portal/visualizacoes/pdf/5916/#/p:1/e:5916

# Funciona o download da forma
#url= "https://ioes.dio.es.gov.br/buscanova/#/diariodaserra/p=1&q=nomear%20para%20o%20cargo"

from bs4 import BeautifulSoup
import requests
import wget


url = "https://ioes.dio.es.gov.br/buscanova/#/diariodaserra"
PARAMS = {'p':1, 'q': 'nomear%20para%20o%20cargo'}
r  = requests.get(url = url, params = PARAMS)
data = r.text
soup = BeautifulSoup(data)

# wget.download("https://ioes.dio.es.gov.br/diariodaserra/portal/edicoes/download/4801")

# ja baixados
# 1500 a 3000
# 2000 a 3000 (interompido)
# 3000 a 5000

for i in range(1500,3000):
    try: 
        wget.download("https://ioes.dio.es.gov.br/diariodaserra/portal/edicoes/download/"+str(i))
   #     nomeArqSaida = 'c:\\temp\\serra_dioes\\' + str(i) + '.pdf'
    except:
        continue


In [ ]:
from PyPDF2 import PdfFileReader
import re
import os

for file in os.listdir("C:\\temp\\serra_dioes"):
    if file.endswith(".pdf"): # and file.startswith("serra"):
        try:
            input = PdfFileReader("C:\\temp\\serra_dioes\\"+file)
            NumPages = input.getNumPages()

            #StringInicio = "NOMEAR"
            #StringFim = "REF."

    # Exemplos de nomecacoes
    # Art. 1° NOMEAR, CLENYA SALES DE SEIXAS, para exercer o cargo de provimento em comissão de Coordenador -REF. IP-03.

    #NOMEAR, com fulcro no artigo 12, inciso II da Lei Complementar n.º 46, de 31 de janeiro de 1994, o Senhor BRUNNO BATISTA
    #CONTARATO, para o cargo de provimento em comissão Assessor Especial, Ref. DER-03,

    #Art. 1º Nomear Fabricia de Souza Barros , para exercer o cargo de provimento Assessor Especial III, do Instituto de Pesos e Medidas do
    #Estado do Espírito Santo - IPEM/ES,

            for pag in range(0, NumPages):
                #print('pagina ' , pag)
                PageObj = input.getPage(pag)
                Text = PageObj.extractText()

                #expRegular = "NOMEAR(.*?)CARGO(.*?)(\s*)REF." 
                #expRegular = "NOMEAR(.*?)CARGO(.*?)(\n\n)" 
                expRegular = "NOMEAR(.*?)CARGO(.*?)REF."
                
                expRegular2 = "Nomeia(.*?)Art."

                x1 = re.search(expRegular, Text, flags=(re.DOTALL | re.IGNORECASE )) 
                x2 = re.search(expRegular2, Text, flags=(re.DOTALL | re.IGNORECASE )) 
                
                if(x1 is not None):
                    print(x1.group(0))
                    arqtxt = "C:\\temp\\serra_dioes\\"+str(file)+"_"+str(pag)+".txt"
                    if not os.path.exists(arqtxt):
                        file = open(arqtxt, "w",encoding='utf-8') 
                        print('criando arquivo ' + arqtxt)    
                        file.write(x1.group()) 
                        file.close() 
                        
                        
                if(x2 is not None):
                    print(x2.group(0))
                    arqtxt = "C:\\temp\\serra_dioes\\"+str(file)+"_"+str(pag)+".txt"
                    if not os.path.exists(arqtxt):
                        file = open(arqtxt, "w",encoding='utf-8') 
                        print('criando arquivo ' + arqtxt)    
                        file.write(x2.group()) 
                        file.close() 
                        
        except Exception as e:
            print(e)
            continue

In [ ]:
# Serra - ES
# Com API Fitz

import fitz 
import re
import os


for file in os.listdir("C:\\temp\\serra_dioes"):
    if file.endswith(".pdf"): # and file.startswith("serra"):
        try:
            doc = fitz.open("C:\\Temp\\serra_dioes\\"+file)
            text_list=[] 
            for i in range(doc.pageCount):  
                pag = i
                pg=doc.loadPage(i) 
                Text=pg.getText() 

                expRegular = "NOMEAR(.*?)CARGO(.*?)REF."
                expRegular2 = "Nomeia(.*?)Art."

                x1 = re.search(expRegular, Text, flags=(re.DOTALL | re.IGNORECASE )) 
                x2 = re.search(expRegular2, Text, flags=(re.DOTALL | re.IGNORECASE )) 
                
                if(x1 is not None):
                    #print(x1.group(0))
                    arqtxt = "C:\\temp\\serra_dioes\\"+str(file)+"-"+str(pag)+".txt"
                    if not os.path.exists(arqtxt):
                        file = open(arqtxt, "w",encoding='utf-8') 
                        print('criando arquivo ' + arqtxt)    
                        file.write(x1.group()) 
                        file.close() 
                        
                if(x2 is not None):
                    #print(x2.group(0))
                    arqtxt = "C:\\temp\\serra_dioes\\"+str(file)+"_"+str(pag)+".txt"
                    if not os.path.exists(arqtxt):
                        file = open(arqtxt, "w", encoding='utf-8') 
                        print('criando arquivo ' + arqtxt)    
                        file.write(x2.group()) 
                        file.close() 
                        
        except Exception as e:
                    print(e)
                    continue

In [ ]:
# Logica da leitura dos txts

import csv
import re
import os

# NOMEAR, xxxxxx , NOME DA PESSOA , para exercer o cargo de XXXXXXXXXXXX

for file in os.listdir("C:\\temp\\serra_dioes"):
    if file.endswith(".txt"):
        with open("C:\\temp\\serra_dioes\\"+file, 'r',encoding="utf-8") as read_obj:
            try:
                Text = ""
                for line in read_obj:
                    Text = Text + line

                with open('C:\\temp\\serra_dioes\\'+file+'.csv', 'w', newline='') as csvfile:
                    s1 = ""
                    s2 = ""
                    
                    print("################")
                    print(file)

                    # Obtem o nome da pessoa (entre virgulas)
                    
                    # [A-Z\u00C0-\u00FF\s.]+  uma ou mais vezes esse conjunto
                    # A-Z maiusculas
                    # \u00C0-\u00FF  caracteres acentuados
                    # \s espaço em branco
                    # .   aceita pontos
                    
                    # casos onde nao foi possivel
                    # diarios sem as virgulas
                    # diarios com o nome em minuscula
                    
                    expRegular = r",\n{0,}\s{0,}[A-Z\u00C0-\u00FF\s.]+\n{0,}\s{0,},"                     
                    
                    l1 = re.findall(expRegular, Text, flags=(re.DOTALL ))  
                    print(l1)
                    
                    #x1 = re.search(expRegular, Text, flags=(re.DOTALL ))  
                    #    if(x1 is not None):
                    #        s1 = x1.group(0)
                    #        #s1 = str(x1)
                    #        s1 = s1.replace("\n","")
                    #        s1 = s1.replace(", ","")
                    #        s1 = s1.replace(",","")
                    #        print(s1)
                 
                 # tenta novamente com minusculas
                #   if(x1 is None): 
                #        expRegular = r",\n{0,}\s{0,}[a-zA-Z\u00C0-\u00FF\s.]+\n{0,}\s{0,},"
#                        x1 = re.search(expRegular, Text, flags=(re.DOTALL ))  
#                        s1 = x1.group(0)

                        
                    # Obtem o cargo - tudo à direita da expressao "cargo"
                   # expRegular = "(?<=cargo)(?s)(.*$)"
            
                    # Obtem o cargo, parando em um ponto
                    expRegular = "(cargo)(.*?)\."
                    l2 = re.findall(expRegular, Text, flags=(re.DOTALL )) 
                    print(l2)
                    
                  #  x2 = re.search(expRegular, Text, flags=(re.MULTILINE )) 
                 #   if(x2 is not None):
                     #       s2 = x2.group(0)
                     #       s2 = s2.replace("\n","")
                          #  print(s2)
                    
                    
                    print("################")
                        
                    writer = csv.writer(csvfile, delimiter=';')
                    for i in range(len(l1)):
                        print(l1[i])
                        lista = [0,l1[i],l2[i],file,"Serra","ES"]
                        #lista = [0,x1,x2,file,"Serra","ES"]
                        writer.writerow(lista)
            except:
                    print("Excecao " + file)
                    continue


In [ ]:
# Faz merge dos cvs gerados para a Serra - ES
import os

f = open('C:\\temp\\serra_dioes\\CONSOLIDADO.csv', 'w')
for file in os.listdir("C:\\temp\\serra_dioes"):
    ##     if os.path.isfile(os.path.join(root, item)):
    if file.endswith(".csv") and not file.startswith("CONSOLIDADO"):
        with open("C:\\temp\\serra_dioes\\"+file, 'r') as read_obj:
            for line in read_obj:
                f.write(line)
f.close()

In [ ]:
from PyPDF2 import PdfFileReader
import re
import os

for file in os.listdir("C:\\temp\\serra2"):
    if file.endswith(".pdf"): # and file.startswith("serra"):
        try:
            input = PdfFileReader("C:\\temp\\serra2\\"+file)
            NumPages = input.getNumPages()
    
            for pag in range(0, NumPages):
                #print('pagina ' , pag)
                PageObj = input.getPage(pag)
                Text = PageObj.extractText()

                expRegular = "NOMEAR(.*?)CARGO(.*?)REF."
                
                expRegular2 = "Nomeia(.*?)Art."

                x1 = re.search(expRegular, Text, flags=(re.DOTALL | re.IGNORECASE )) 
                x2 = re.search(expRegular2, Text, flags=(re.DOTALL | re.IGNORECASE )) 
                
                if(x1 is not None):
                    print(x1.group(0))
                    arqtxt = "C:\\temp\\serra_2\\"+str(file)+"_"+str(pag)+".txt"
                    if not os.path.exists(arqtxt):
                        file = open(arqtxt, "w",encoding='utf-8') 
                        print('criando arquivo ' + arqtxt)    
                        file.write(x1.group()) 
                        file.close() 
                        
                        
                if(x2 is not None):
                    print(x2.group(0))
                    arqtxt = "C:\\temp\\serra2\\"+str(file)+"_"+str(pag)+".txt"
                    if not os.path.exists(arqtxt):
                        file = open(arqtxt, "w",encoding='utf-8') 
                        print('criando arquivo ' + arqtxt)    
                        file.write(x2.group()) 
                        file.close() 
                        
        except:
            continue


In [ ]:
# Mesma lógica do serra_dioes?

import re
import os
import csv

for file in os.listdir("C:\\temp\\serra2"):
    if file.endswith(".txt"):
        with open("C:\\temp\\serra2\\"+file, 'r',encoding="utf-8") as read_obj:
            try:
                Text = ""
                for line in read_obj:
                    Text = Text + line
                    expRegular = r"Nomeia\s{0,}(.*?)\s{0,}(.*?)\s{0,}(.*?)\s{0,}(.*?)exercer\s{0,}o\s{0,}cargo\s{0,}(.*?)remuneração"                     
                    l1 = re.findall(expRegular, Text, flags=(re.DOTALL ))  
                    if len(l1) > 0:
                        print(l1)
                        with open('C:\\temp\\serra2\\'+file+'.csv', 'w', newline='') as csvfile:
                            writer = csv.writer(csvfile, delimiter=';')
                            for i in range(len(l1)):
                                lista = [0,l1[i][3],l1[i][4],file,"Serra","ES"]
                                writer.writerow(lista)
            except Exception as e:
                    print(e)
                    print("Excecao " + file)
                    continue


In [ ]:
# Faz merge dos cvs gerados para a Serra2
import os

f = open('C:\\temp\\serra2\\CONSOLIDADO.csv', 'w')
for file in os.listdir("C:\\temp\\serra2"):
    ##     if os.path.isfile(os.path.join(root, item)):
    if file.endswith(".csv") and not file.startswith("CONSOLIDADO"):
        with open("C:\\temp\\serra2\\"+file, 'r') as read_obj:
            for line in read_obj:
                f.write(line)
f.close()